# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8f7884a280>
├── 'a' --> tensor([[-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181]])
└── 'x' --> <FastTreeValue 0x7f8f7884a310>
    └── 'c' --> tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                        [-0.4436, -2.7683,  0.6299,  0.1708],
                        [ 0.6388,  0.3368,  1.3827,  0.1359]])

In [4]:
t.a

tensor([[-0.2674,  2.6716,  0.1978],
        [-0.2657, -0.7307, -1.7181]])

In [5]:
%timeit t.a

63.4 ns ± 0.038 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8f7884a280>
├── 'a' --> tensor([[ 0.0695,  0.7286,  0.8693],
│                   [-2.1787, -1.0719,  2.0632]])
└── 'x' --> <FastTreeValue 0x7f8f7884a310>
    └── 'c' --> tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                        [-0.4436, -2.7683,  0.6299,  0.1708],
                        [ 0.6388,  0.3368,  1.3827,  0.1359]])

In [7]:
%timeit t.a = new_value

66.1 ns ± 0.0446 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2674,  2.6716,  0.1978],
               [-0.2657, -0.7307, -1.7181]]),
    x: Batch(
           c: tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359]]),
       ),
)

In [10]:
b.a

tensor([[-0.2674,  2.6716,  0.1978],
        [-0.2657, -0.7307, -1.7181]])

In [11]:
%timeit b.a

57.3 ns ± 0.148 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0436, -1.1860, -1.4013],
               [-0.2937,  0.0066, -0.1546]]),
    x: Batch(
           c: tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.803 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

844 ns ± 0.627 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 38.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 721 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8f787d5f10>
├── 'a' --> tensor([[[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]],
│           
│                   [[-0.2674,  2.6716,  0.1978],
│                    [-0.2657, -0.7307, -1.7181]]])
└── 'x' --> <FastTreeValue 0x7f8ed5196610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 154 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8ece4c6130>
├── 'a' --> tensor([[-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181],
│                   [-0.2674,  2.6716,  0.1978],
│                   [-0.2657, -0.7307, -1.7181]])
└── 'x' --> <FastTreeValue 0x7f8f787d0340>
    └── 'c' --> tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                        [-0.4436, -2.7683,  0.6299,  0.1708],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 43.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 79 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.3459,  1.7478,  1.3769, -0.1486],
                       [-0.4436, -2.7683,  0.6299,  0.1708],
                       [ 0.6388,  0.3368,  1.3827,  0.1359]],
              
                      [[-0.3459,  1.7478,  1.3769, -0.1486],
                       [-0.4436, -2.7683,  0.6299,  0.1708],
                       [ 0.6388,  0.3368,  1.3827,  0.1359]],
              
                      [[-0.3459,  1.7478,  1.3769, -0.1486],
                       [-0.4436, -2.7683,  0.6299,  0.1708],
                       [ 0.6388,  0.3368,  1.3827,  0.1359]],
              
                      [[-0.3459,  1.7478,  1.3769, -0.1486],
                       [-0.4436, -2.7683,  0.6299,  0.1708],
                       [ 0.6388,  0.3368,  1.3827,  0.1359]],
              
                      [[-0.3459,  1.7478,  1.3769, -0.1486],
                       [-0.4436, -2.7683,  0.6299,  0.1708],
                       [ 0.6388,  0.3368,  1.3827,  0.1359]],

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 341 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359],
                      [-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359],
                      [-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359],
                      [-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359],
                      [-0.3459,  1.7478,  1.3769, -0.1486],
                      [-0.4436, -2.7683,  0.6299,  0.1708],
                      [ 0.6388,  0.3368,  1.3827,  0.1359],
                      [-0.3459,  1.7478,  1.3769, -0.1486],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 333 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 815 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
